### This function operates on image files and  renames files numerically and can also resize the images
#### If you download images files from an internet search you will notice the files often have very long 
#### and un-weildy names often with strange characters in the name. Also these files may come in a 
#### multitude of image formats as denotd by their extensions. After I download these images I find it
#### convenient to rename the files and generally to convert them all to a common image format.
#### This function achieves that purpose. It will nename the files in a numerical order starting with a 
#### number you specificy. 
#### You specify a new file extension to converted all files to the image format specified by the extension.
#### Alternatively you can specify parameter new_ext='same' to preserve the original extension.
####  You can also elect to risize all the images by specifying the parameters height and width.
####  File are nenamed in a numerical sequence starting with the value assigned to parameter snum.
####  Note- files are rename using "zeros padding" which preserves the order that files are processed
#### in for python functions.  Unlike windows, files in most python functions read files alphanumerically.
#### For example is you have files in a directory with names 1.jpg, 2.jpg, ... 9.jpg, 10,jpg the order in which
#### the will be processed is not the numeric order but the alphabetic order. Thus they will be 
#### processed in the order 1.jpg, 10.jpg, 2.jpg etc. The use of "zeros' padding will prevent this.
#### With zero padding the files are renamed 01.jpg, 02.jpg .. 09.jpg, 10.jpg so the are processed in
#### the order you would expect.
### Use:
#####    1- place the images you want processed in a single directory. I call it the source_dir.  place ONLY image files in the directory.
#####     NOTE this function use the cv2 module so   file types are limited to having extensions 'jpg', 'jpe', 'jpeg', 'png', 'bmp', 'tiff', 'jfif' .
#####     2 create a python environment with the module cv2 (OpenCV) and tqdm installed
#####     3 The function call is of the form rename (source_dir, snum=1, new_ext='same', height=None, width=None) where:
#####         dir is a string denoting the full path to the directory containing the image files
#####         snum is an integer (default value=1) that defines the starting number for the numerical renaming of the files
#####         new_ext is a string (default= 'same) denoting the file format all image files will be  converted to. 
#####        If set to 'same' the original extensions will be preserved
#####       height is an integer specify the new height of all images - defualts to None - meaning no resizing of the images
#####       width is an integer specify the new width of all images - defualts to None - meaning no resizing of the images
            
#### Processing:
#####      1 The function first checks the input parameters and prints out a message if errors are detected
#####      2 Next the files in dir are checked to verify the file extensions are in the list of acceptable extensions
#####         as defined in the list above. If files are found with invalid extensions, a sub directory called
#####         "bad_ext" is created within the source_dir and the file with the improper extension is removed from
#####          source_dir and stored in the sub directory. A bad_list is created and the filenameshaving improper
#####          extensions are stored in that list and provided as part of the functions return tupple.
#####       3 Next each of the remaining files in dir are processed sequentially. CV2 is used  to read in the image file.
#####          If an error is encounter the image file is corrupted and can not be processed.The file name is added 
#####          to the bad_list and the file is  removed from the source_dir and placed into the "bad_ext subdirectory
#####       4 Now the source_dir directory only  contains image files that can be processed. These files are then 
#####           renamed sequentially start from snum and if parameters height and width are specified the image
#####           is resized. If parameter new_ext is not set to 'same' then the files are written back to the source_dir
#####           with the new extension you specified

#### returns: 
#####        The function returns a tupple of the form (status, bad_list) where:
#####       status is a boolean. It is set to true if at least one image file was  processed to completetion.
#####        Otherwise it is set to False
#####        bad_list is a list of file names. If files in dir were found to have  extensions not in the list above,
#####        or if an image file could not be read by cv2  the file name is included in the list. 
#####       These files will be stored in the   sub directory bad ext in the source_dir

#### Import needed modules

In [1]:
import os
import shutil 
import cv2
from tqdm import tqdm

#### The function below checks the input parameters for errors and files for extension types

In [2]:
def check_dir(source_dir, ext_list, n_ext, height, width):
    f_list=[]  # list to hold acceptable files
    bad_list=[] # list to hold file names with improper extensions or corrupted image files
    one_good=False  # set to true if at least one good image was processed and saved
    processed=0 # initialize count of files processed
    save_path=os.path.join(source_dir,'bad ext') # create path to store the bad  files
    if os.path.isdir(source_dir)==False: # check if source directory contains files
        print('\n The directory {0} does not exist, program  is terminating'.format(source_dir))
        return (False, None, None, processed)
    mod_list=['jpg', 'jpe', 'jpeg', 'png', 'bmp', 'tiff']
    if n_ext not in mod_list and n_ext !='same': # make sure new extension if specified is in acceptable list
        msg='\nthe new extension you specified {0} is not in the list of acceptable extensions '
        print(msg.format(n_ext))
        print('The list of valid extensions is ', mod_list, ' **** program terminated ****')
        return (False, None, None, processed)
    if ( height !=None and width==None) or (height ==None and width != None): # if resizing make sure both height and width are specified
        print(' *** ERROR *** you must specify both the height and width to resize the images - Program Terminating')
        return (False, None, None, processed)
    else:
        dir_list=os.listdir(source_dir) # create a list of files in the source_dir
        if len(dir_list)==0: # check if there are files in the source_dir
            print('\ndirectory {0} is empty, program is terminating'.format(source_dir))
            return (False,None,None, processed)
        else:            
            for d in tqdm(dir_list): # iterate through the files
                f_path=os.path.join(source_dir,d) # path to the file
                if os.path.isfile(f_path): # check that it is a file and not a directory
                    processed=processed +1 # increment files processed counter
                    index=f_path.rfind('.')  # find the location of the last . character                      
                    ext=f_path[index+1:] # this is the file extension like jpg, bmp etc
                    if ext=="jfif": # a jfif extension is the same as a jpg extension
                        ext='jpg'   # rename the files extension as jpg
                        fname=f_path[:index] # store the file name without the  original extension
                        new_path=fname + '.jpg'  # rename the file with a .jpg extension                      
                        shutil.copy(f_path,new_path) # rename the file in the source directory
                        os.remove(f_path)  # remove the original file from the source directory                      
                        f_path=new_path
                    if ext.lower() in ext_list: # for the current file check if its extension is in the extension list
                        # extensions are valid but are they really image files?
                        # try to read in each file to test it
                        try:
                            img= cv2.imread(f_path) # try to read in the image
                            imsize=img.size # if the image was not read this line will create an exception
                            if height !=None and width !=None:
                                img=cv2.resize(img, (width, height))   # resize the image                             
                            cv2.imwrite( f_path, img) # write the image back to the source_dir
                            f_list.append(f_path)  # add this this file path to list of good images
                            one_good=True  # note there is at lead one valid image file                                                      
                        except(AttributeError ):
                            # file is not a valid image file
                            msg='\nfile {0} is not a valid image file, it wil be removed and saved to {1}'
                            print(msg.format(f_path, save_path ))
                            bad_list.append(f_path) # append the file path to the list of bad images
                    else:
                        bad_list.append(f_path) # extension of the file was not in the list off acceptable file types
        if one_good==False:
            print('There are no files in {0} that have the proper image extensions'.format(dir))
            return (False, None, bad_list, processed)
        else:
            if len(bad_list)==0: # check if all files were good
                return (True,f_list, None, processed)
            else:   # there are some bad files in the source directory              
                #msg='\n {0} files in {1} have invalid extensions or are not valid image files and will be moved to {2}'
                #print(msg.format(len(bad_list), dir, save_path))
                if os.path.isdir(save_path)==False:
                    os.mkdir(save_path)  # make a directory to store the bad image files
                for f in bad_list:
                    # extract the file name
                    fsplit=f.split('\\')                    
                    fname=fsplit[len(fsplit)-1]                    
                    test_path=os.path.join(save_path,fname)                    
                    if os.path.isfile(test_path)==True:
                        os.remove(f_path)
                    else:
                        dest = shutil.move(f, save_path)    # store the bad file to the save_dir                
                msg='\n{0} files have been removed from {1} and stored in {2}'
                print(msg.format(len(bad_list),source_dir, save_path ))
                return (True, f_list, bad_list, processed)

This is the main function

In [3]:
def rename (source_dir, snum=1, new_ext='same', height=None, width=None):
    f_dict={}
    if new_ext !=None:
        new_ext=new_ext.lower()
    ext_list=['jpg', 'jpe', 'jpeg', 'png', 'bmp', 'tiff', 'jfif'] # list of acceptable file types
    status, f_list, bad_list, processed=check_dir(source_dir, ext_list, new_ext, height, width) # run a set of checks on input data
    if status==False:
        return (status, bad_list)    
    else:  #determine the amount of preceeding zeros to pad the file name with to preserve order        
        pad=0
        mod = 10
        fc=len(f_list)  # determine number of goodfiles toprocess      
        for i in range(1, fc + 1): # skip i=0 because 0 modulo anything is 0 and we don't want to increment pad
            if i % mod == 0:
                pad=pad+1                    
                mod =mod * 10
        #  make all file names unique by adding a numeric extension in place of real extension
        for i in range(0,fc):
            f=f_list[i] 
            f_path=os.path.join(source_dir,f) #full path to the file
            index=f.rfind('.')
            fname=f_path[:index]
            ext=f[index:]   # this includes the period (.)
            if ext=='.jfif':  # jfif is the same as a jpg file but cv2 and other modules don't process jfif files
                ext='.jpg'
            #to avoid trying to rename a file with a name that is in the directory
            # give each file a new unique numerical extension
            #sore the original extension in a dictionay where the key is the file name
            fnew=fname +'.' + str(i+1)
            new_path=os.path.join(source_dir, fnew)            
            os.rename(f_path, new_path)
            f_dict[fname]=ext  #key is new file name, value is original extension
            #now all full filenames are unique with unique extensions
        f_list=os.listdir (source_dir)
        i=0
        good=0
        for d in tqdm(f_list):
            d_path=os.path.join (source_dir,d)
            if os.path.isdir(d_path)==False:                
                index=d_path.rfind('.')
                fname=d_path[:index]               
                ext=f_dict[fname]                
                fname=str(i + snum).zfill(pad+1) + ext
                i=i+1
                new_path=os.path.join(source_dir, fname)
                shutil.copy(d_path,new_path)                
                os.remove(d_path)                
                if new_ext !=None:                                        
                    index=new_path.rfind('.')
                    old_ext=new_path[index+1:]
                    if new_ext !='same':
                        fname=new_path[:index]
                        new_name=fname + '.' + new_ext
                        img= cv2.imread(new_path)
                        cv2.imwrite( new_name, img)                        
                        os.remove(new_path)
        if new_ext=='same':            
            msg='{0} files were processed {1} files were renamed and saved with the original extensions'.format(processed,fc)
        else:
             msg='{0} files were processed {1} files and saved with new extension {2}'.format(processed,fc, new_ext) 
        print (msg)
        return (True, bad_list)

In [4]:
status, bad_list=rename(r'C:\Temp\rename tests')                                                      

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 208.35it/s]

10 files were processed 10 files were renamed and saved with the original extensions
